In [1]:
import sys
sys.path.append('../..')
sys.path.append('../../APDFT')
sys.path.append('../../helper_code')
sys.path.append('../data')

import pickle
from pyscf import gto, scf, dft, cc
from ase import Atoms
import numpy as np
import pandas as pd
import pyscf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import basis_set_exchange as bse
from APDFT.FcMole import *
import os
import ast
from copy import deepcopy
from IPython.display import display
from helper_code.data_processing import *
import qml
from helper_code.util import charge_arr_to_str
from ase import Atoms
from ase.io import write

%load_ext autoreload
%autoreload 2
from APDFT.AP_class import APDFT_perturbator as AP

In [2]:
coronene_energy_raw_data = np.load("../data/coronene_raw_data/coronene_mutants_pbe0_pcx2.npz", allow_pickle=True)
charges, total_energy = coronene_energy_raw_data['charges'], coronene_energy_raw_data['energies']

columns = ['charges', 'total energy']
coronene_energy_data = pd.DataFrame(columns=columns)
coronene_energy_data['charges'] = charges.tolist()
coronene_energy_data['total energy'] = total_energy.tolist()

CORONENE_REF_ENERGY = -909.721935153841
coronene_energy_data['delta total energy'] = coronene_energy_data['total energy'].apply(lambda x: CORONENE_REF_ENERGY-x)

display(coronene_energy_data.head(3))

,charges,total energy,delta total energy
0,"[7, 5, 5, 5, 5, 7, 5, 7, 5, 5, 7, 7, 5, 7, 6, ...",-946.036986,36.315051
1,"[6, 5, 7, 6, 6, 6, 6, 6, 6, 6, 5, 7, 6, 5, 7, ...",-929.442623,19.720687
2,"[6, 5, 7, 5, 5, 5, 7, 5, 5, 6, 5, 7, 7, 5, 6, ...",-942.508112,32.786177


In [3]:
xyz_file = '../data/coronene_raw_data/coronene_opt_pbe0_pcX2.xyz'
with open(xyz_file, 'r') as f:
    lines = f.readlines()[2:]  # Skip the first two lines (atom count and comment)
    atoms = [line.split() for line in lines]

# Extract atomic symbols and coordinates
symbols = [atom[0] for atom in atoms]
all_atoms_coord = [[float(atom[1]), float(atom[2]), float(atom[3])] for atom in atoms]

In [4]:
sample_charge_arr = coronene_energy_data['charges'][0]
compressed_str = charge_arr_to_str(sample_charge_arr)
ase_mol = Atoms(symbol=compressed_str, position=all_atoms_coord)
write("temp_mol.xyz", ase_mol, format='xyz')
mol = qml.Compound(xyz="temp_mol.xyz")
mol.generate_coulomb_matrix(size=36, sorting="row-norm")
CM_rep = mol.representation
print(CM_rep)
os.remove("temp_mol.xyz")

AttributeError: 'str' object has no attribute 'tolist'